In [1]:
import os
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.layers import (Dense, 
                                     BatchNormalization, 
                                     LeakyReLU, 
                                     Reshape, 
                                     Conv2DTranspose,
                                     Conv2D,
                                     Dropout,
                                     Flatten,
                                     multiply)

import tensorflow_addons as tfa

from tensorflow.keras.applications import EfficientNetB0
#from tensorflow.keras.applications import EfficientNetB7
from keras.models import Model
from keras.models import Sequential
import efficientnet.tfkeras
import efficientnet.keras as efn 

In [2]:
"""
This version uses the entire EfficientNet model with conv2dtranspose for upscaling the output of the pretrained network to the same dimensions as the input image.
"""

def efficientnet():
    model = efn.EfficientNetB3(
        include_top=False,
        weights="imagenet",
        input_shape=(256, 256, 3))
    
    # add new classifier layers
    x = layers.Conv2DTranspose(filters=640, kernel_size=(2, 2), strides=(2, 2), use_bias=False)(model.layers[-36].output)

    x = tfa.layers.InstanceNormalization()(x)
    x = layers.Conv2DTranspose(filters=160, kernel_size=(4, 4), strides=(4, 4), use_bias=False)(x)
    x = tfa.layers.InstanceNormalization()(x)
    x = layers.Conv2DTranspose(filters=40, kernel_size=(2, 2), strides=(2, 2), use_bias=False)(x)
    x = tfa.layers.InstanceNormalization()(x)
    x = layers.Conv2DTranspose(filters=3, kernel_size=(2, 2), strides=(2, 2), use_bias=False)(x)
    
    # define new model
    model = Model(inputs=model.inputs, outputs=x)

    model.compile()
    model.summary()
    return model

In [3]:
efficientnet()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 128, 128, 40) 1080        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 128, 128, 40) 160         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 128, 128, 40) 0           stem_bn[0][0]                    
_______________________________________________________________________________________

In [5]:
def efficientnet_generator():
    efficientnet_model = efn.EfficientNetB3(
        include_top=False,
        weights="imagenet",
        input_shape=(256, 256, 3))

    """
    Here I am trying to use a middle portion of the model. This doesn't work as there is an issue with the squeeze and excite layers. For some reason they only work      if you provide the entire model, or cut off from the top, but cannot cut off from the bottom (layers 1-30 for example). These layers have to be removed and done      manually but the performance is then not as good as the pretrained weights are no longer being used.
    """

    inputs = tf.keras.layers.Input(shape=[256,256,96])

    x=efficientnet_model.layers[71](inputs)


    #x=efficientnet_model.layers[29](inputs)

    """
    for layer in efficientnet_model.layers[30:114]:
        x=layer(x)
    """
    return x


In [ ]:
def efficientnet_generator():
    efficientnet_model = efn.EfficientNetB3(
        include_top=False,
        weights="imagenet",
        input_shape=(256, 256, 3))

    inputs = tf.keras.layers.Input(shape=[256,256,96])

    m = Sequential()
    for layer in efficientnet_model.layers[42:69]:
        m.add(layer)
    
    m.compile()
    m.fit()
    m.summary()
    return m



In [ ]:
import random
def split(model, start, end, name):    
    confs = model.get_config()
    kept_layers = set()
    for i, l in enumerate(confs['layers']):
        if i == 0:
            confs['layers'][0]['config']['batch_input_shape'] = model.layers[start].input_shape
            if i != start:
                confs['layers'][0]['name']
                confs['layers'][0]['config']['name'] = confs['layers'][0]['name']
        elif i < start or i > end:
            continue
        kept_layers.add(l['name'])
    # filter layers
    layers = [l for l in confs['layers'] if l['name'] in kept_layers]
    layers[1]['inbound_nodes'][0][0][0] = layers[0]['name']
    # set conf
    confs['layers'] = layers
    confs['input_layers'][0][0] = layers[0]['name']
    confs['output_layers'][0][0] = layers[-1]['name']
    # create new model
    submodel = Model.from_config(confs)
    for l in submodel.layers:
        orig_l = model.get_layer(l.name)
        if orig_l is not None:
            l.set_weights(orig_l.get_weights())
    
    for layer in submodel.layers:
        layer._name = layer.name + str(name)

    return submodel

In [ ]:
def efficient_block(inputs, name):

    model = efn.EfficientNetB3(
        include_top=False,
        weights="imagenet",
        input_shape=(256, 256, 3))
    
    block = split(model, 30, 71)(inputs)
    x = layers.Conv2DTranspose(144, kernel_size=(1,1), strides=(1,1), padding="same", use_bias=False)(block)

    return x

In [ ]:
def efficient_block(input, name):
    model = efn.EfficientNetB3(
        include_top=False,
        weights="imagenet",
        input_shape=(256, 256, 3))

    block = split(model, 30, 71)
    x = layers.Conv2DTranspose(144, kernel_size=(1,1), strides=(1,1), padding="same", use_bias=False)(block)
    
    for layer in block.layers:
        layer._name = layer.name + str(name)

    return x

In [8]:
def test_layer():
    efficientnet_model = efn.EfficientNetB3(
        include_top=False,
        weights="imagenet",
        input_shape=(256, 256, 3))

    print(efficientnet_model.layers[71].name)  

In [9]:
test_layer()

block3a_expand_activation


In [ ]:
class ReflectionPadding2D(layers.Layer):
    """Implements Reflection Padding as a layer.

    Args:
        padding(tuple): Amount of padding for the
        spatial dimensions.

    Returns:
        A padded tensor with the same type as the input tensor.
    """


    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def call(self, input_tensor, mask=None):
        padding_width, padding_height = self.padding
        padding_tensor = [
            [0, 0],
            [padding_height, padding_height],
            [padding_width, padding_width],
            [0, 0],
        ]
        return tf.pad(input_tensor, padding_tensor, mode="REFLECT")


In [ ]:
def efficient_block(x, name):

    efficientnet_model = efn.EfficientNetB3(
            include_top=False,
            weights="imagenet",
            input_shape=(256, 256, 3)
            )

    inputs = x
    
    for layer in efficientnet_model.layers:
        layer._name = layer.name + str(name)
    
    #efficientnet_model.trainable = False
    
    x = efficientnet_model.layers[30](inputs)
   
    x1 = efficientnet_model.layers[31](x)
    x = efficientnet_model.layers[32](x1)
    x = efficientnet_model.layers[33](x)
    x = efficientnet_model.layers[34](x)
    x2 = efficientnet_model.layers[35](x)
    x = tf.keras.layers.Multiply()([x1, x2])

    x = efficientnet_model.layers[37](x)
    x5 = efficientnet_model.layers[38](x)
    x = efficientnet_model.layers[39](x5)
    x = efficientnet_model.layers[40](x)
    x = efficientnet_model.layers[41](x)
    x = efficientnet_model.layers[42](x)
    x = efficientnet_model.layers[43](x)
    
    x3 = efficientnet_model.layers[44](x)
    x = efficientnet_model.layers[45](x3)
    x = efficientnet_model.layers[46](x)
    x = efficientnet_model.layers[47](x)
    x4 = efficientnet_model.layers[48](x)
    x = tf.keras.layers.Multiply()([x3, x4])
    
    x = efficientnet_model.layers[50](x)
    x = efficientnet_model.layers[51](x)

    x6 = efficientnet_model.layers[52](x)
    x9 = tf.keras.layers.Add()([x5, x6])
    
    x = efficientnet_model.layers[54](x9)
    x = efficientnet_model.layers[55](x)
    x = efficientnet_model.layers[56](x)
    x = efficientnet_model.layers[57](x)
    x = efficientnet_model.layers[58](x)
    
    x7 = efficientnet_model.layers[59](x)
    x = efficientnet_model.layers[60](x7)
    x = efficientnet_model.layers[61](x)
    x = efficientnet_model.layers[62](x)
    x8 = efficientnet_model.layers[63](x)
    x = tf.keras.layers.Multiply()([x7, x8])
    
    x = efficientnet_model.layers[65](x)
    x = efficientnet_model.layers[66](x)
    x10 = efficientnet_model.layers[67](x)
    x = tf.keras.layers.Add()([x9, x10])
    
    x = efficientnet_model.layers[69](x)
    x = efficientnet_model.layers[70](x)
    x = efficientnet_model.layers[71](x)
    x = layers.Conv2DTranspose(144, kernel_size=(1,1), strides=(1,1), padding="same", use_bias=False)(x)
    
    #model = keras.models.Model(inputs, x, name=name)
    """
    x=efficientnet_model.layers[29](x)
    for layer in efficientnet_model.layers[30:60]:
        x=layer(x)
    """
    return x

In [ ]:
def efficientnet_generator(
    filters = 64,
    name=None,
):

    inputs = tf.keras.layers.Input(shape=[256,256,3])

    x = ReflectionPadding2D(padding=(3, 3))(inputs)
    x = layers.Conv2D(filters, (7, 7), use_bias=False)(x)
    x = tfa.layers.InstanceNormalization()(x)
    x = layers.Activation("relu")(x)

    #downsample
    x = layers.Conv2D(64, kernel_size=(3,3), strides=(2,2), padding="same", use_bias=False)(x)
    x = tfa.layers.InstanceNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(144, kernel_size=(3,3), strides=(2,2), padding="same", use_bias=False)(x)
    x = tfa.layers.InstanceNormalization()(x)
    x = layers.Activation("relu")(x)

    #efficient blocks

    #for _ in range(num_efficient_blocks):
    x = efficient_block(x, 'block1')
    x = efficient_block(x, 'block2')
    x = efficient_block(x, 'block3')
    x = efficient_block(x, 'block4')
    x = efficient_block(x, 'block5')
    x = efficient_block(x, 'block6')
    #x = efficient_block(x, 'block7')
    #x = efficient_block(x, 'block8')
    #x = efficient_block(x, 'block9')


    # Upsampling
    x = layers.Conv2DTranspose(64, kernel_size=(3,3), strides=(2,2), padding="same", use_bias=False)(x)
    x = tfa.layers.InstanceNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2DTranspose(3, kernel_size=(3,3), strides=(2,2), padding="same", use_bias=False)(x)
    x = tfa.layers.InstanceNormalization()(x)
    x = layers.Activation("relu")(x)

    # Final block
    x = ReflectionPadding2D(padding=(3, 3))(x)
    x = layers.Conv2D(3, (7, 7), padding="valid")(x)
    x = layers.Activation("tanh")(x)

    model = keras.models.Model(inputs, x, name=name)
    
    model.compile()
    model.summary()
    return model

In [ ]:
efficientnet_generator(
    filters = 64,
    name=None,
)